# 📧 Email Analysis Dashboard - ICE Pipeline

This notebook provides a comprehensive view of all processed emails in the ICE Investment Context Engine pipeline.

## Features:
- 📊 **Email Processing Statistics**
- 🎯 **Investment Entity Analysis** 
- 🕸️ **Knowledge Graph Visualization**
- 📈 **Sentiment Trends**
- 🔍 **Interactive Email Search**
- 💭 **Query Interface**

---

In [1]:
# Setup and imports
import sys
import os
import sqlite3
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import networkx as nx
from datetime import datetime, timedelta
import warnings
import subprocess
import threading
import time
import getpass
import glob
warnings.filterwarnings('ignore')

# Add pipeline components to path
pipeline_path = os.getcwd()
if pipeline_path not in sys.path:
    sys.path.append(pipeline_path)

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

class EmailDashboard:
    """Dashboard class for loading and analyzing processed email data"""
    
    def __init__(self):
        self.emails_df = pd.DataFrame()
        self.attachments_df = pd.DataFrame()
        self.ice_data = []
        self.working_dirs = []
    
    def find_pipeline_databases(self):
        """Find all pipeline state databases in the system"""
        search_patterns = [
            "./pipeline_state.db",
            "./**/pipeline_state.db",
            "/tmp/*/pipeline_state.db",
            "/var/folders/*/pipeline_state.db"
        ]
        
        databases = []
        for pattern in search_patterns:
            databases.extend(glob.glob(pattern, recursive=True))
        
        # Also check for temporary directories from pipeline runs
        temp_dirs = [d for d in os.listdir('/tmp') if d.startswith('real_pipeline_') or d.startswith('pipeline_')]
        for temp_dir in temp_dirs:
            db_path = os.path.join('/tmp', temp_dir, 'pipeline_state.db')
            if os.path.exists(db_path):
                databases.append(db_path)
        
        return databases
    
    def load_emails_from_database(self, db_path):
        """Load email data from SQLite database"""
        try:
            conn = sqlite3.connect(db_path)
            
            # Load emails table
            emails_query = "SELECT * FROM emails"
            emails = pd.read_sql_query(emails_query, conn)
            
            # Load attachments table if exists
            try:
                attachments_query = "SELECT * FROM attachments"
                attachments = pd.read_sql_query(attachments_query, conn)
            except pd.io.sql.DatabaseError:
                attachments = pd.DataFrame()
            
            conn.close()
            
            return emails, attachments
            
        except Exception as e:
            print(f"⚠️ Error loading database {db_path}: {e}")
            return pd.DataFrame(), pd.DataFrame()
    
    def load_ice_data_from_directory(self, working_dir):
        """Load ICE integration data from working directory"""
        ice_data = []
        
        ice_storage_dir = os.path.join(working_dir, 'ice_storage')
        if os.path.exists(ice_storage_dir):
            # Look for ICE result files
            result_files = glob.glob(os.path.join(ice_storage_dir, "**/*.json"), recursive=True)
            
            for file_path in result_files:
                try:
                    with open(file_path, 'r') as f:
                        data = json.load(f)
                        ice_data.append(data)
                except Exception as e:
                    continue
        
        return ice_data
    
    def load_all_data(self):
        """Load all available email processing data"""
        print("🔍 Searching for processed email data...")
        
        databases = self.find_pipeline_databases()
        
        if not databases:
            print("📭 No pipeline databases found")
            return False
        
        print(f"📊 Found {len(databases)} pipeline database(s)")
        
        all_emails = []
        all_attachments = []
        
        for db_path in databases:
            working_dir = os.path.dirname(db_path)
            self.working_dirs.append(working_dir)
            
            print(f"📂 Loading data from: {working_dir}")
            
            # Load email data
            emails, attachments = self.load_emails_from_database(db_path)
            
            if not emails.empty:
                all_emails.append(emails)
                print(f"   📧 {len(emails)} emails")
            
            if not attachments.empty:
                all_attachments.append(attachments)
                print(f"   📎 {len(attachments)} attachments")
            
            # Load ICE data
            ice_data = self.load_ice_data_from_directory(working_dir)
            self.ice_data.extend(ice_data)
            
            if ice_data:
                print(f"   🧠 {len(ice_data)} ICE integrations")
        
        # Combine all data
        if all_emails:
            self.emails_df = pd.concat(all_emails, ignore_index=True)
            # Remove duplicates based on email_uid
            if 'email_uid' in self.emails_df.columns:
                self.emails_df = self.emails_df.drop_duplicates(subset=['email_uid'])
        
        if all_attachments:
            self.attachments_df = pd.concat(all_attachments, ignore_index=True)
            # Remove duplicates
            if 'attachment_id' in self.attachments_df.columns:
                self.attachments_df = self.attachments_df.drop_duplicates(subset=['attachment_id'])
        
        has_data = not self.emails_df.empty or bool(self.ice_data)
        
        if has_data:
            print(f"✅ Total loaded: {len(self.emails_df)} emails, {len(self.attachments_df)} attachments, {len(self.ice_data)} ICE results")
        
        return has_data

print("📊 Email Analysis Dashboard - Ready!")
print(f"📁 Working directory: {pipeline_path}")
print(f"⏰ Dashboard loaded at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

📊 Email Analysis Dashboard - Ready!
📁 Working directory: /Users/royyeo/Library/CloudStorage/OneDrive-NationalUniversityofSingapore/Capstone Project/imap_email_ingestion_pipeline
⏰ Dashboard loaded at: 2025-09-03 20:38:03


## 🔍 Step 1: Load Processed Email Data or Run Pipeline

Let's check for processed email data. If none exists, we'll help you run the pipeline.

In [2]:
def run_pipeline_with_monitoring(password):
    """Run the email processing pipeline with real-time output monitoring"""
    print("🚀 Starting Email Processing Pipeline...")
    print("=" * 60)
    
    try:
        # Start the pipeline process
        process = subprocess.Popen(
            [sys.executable, "process_emails.py", password],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            universal_newlines=True,
            bufsize=1
        )
        
        # Monitor output in real-time
        output_lines = []
        while True:
            line = process.stdout.readline()
            if not line and process.poll() is not None:
                break
            if line:
                print(line.rstrip())
                output_lines.append(line.rstrip())
        
        # Get the return code
        return_code = process.poll()
        
        if return_code == 0:
            print("\n🎉 Pipeline completed successfully!")
            return True, output_lines
        else:
            print(f"\n❌ Pipeline failed with return code: {return_code}")
            return False, output_lines
            
    except Exception as e:
        print(f"❌ Error running pipeline: {e}")
        return False, []

def test_email_connection(password):
    """Test email connection before running full pipeline"""
    print("🔍 Testing email connection...")
    
    try:
        process = subprocess.run(
            [sys.executable, "quick_email_test.py", password],
            capture_output=True,
            text=True,
            timeout=120  # 2 minute timeout for connection test
        )
        
        print(process.stdout)
        if process.stderr:
            print(process.stderr)
            
        return process.returncode == 0
        
    except subprocess.TimeoutExpired:
        print("⏰ Connection test timed out after 2 minutes")
        return False
    except Exception as e:
        print(f"❌ Connection test error: {e}")
        return False

def is_interactive_environment():
    """Check if we're running in an interactive environment"""
    try:
        # Check if we can get input (works in interactive environments)
        import sys
        return hasattr(sys.stdin, 'isatty') and sys.stdin.isatty()
    except:
        return False

def get_password_from_environment():
    """Try to get password from environment variable for non-interactive execution"""
    return os.environ.get('EMAIL_PASSWORD', '')

# Initialize dashboard
dashboard = EmailDashboard()
has_data = dashboard.load_all_data()

if not has_data:
    print("\n⚠️ No processed email data found.")
    
    # Check if we're in an interactive environment
    is_interactive = is_interactive_environment()
    
    if is_interactive:
        print("🔄 Let's run the email processing pipeline to get started!")
        print()
        
        # Get user password securely in interactive mode
        try:
            run_pipeline = input("Would you like to run the email processing pipeline? (y/n): ").lower().strip()
            
            if run_pipeline in ['y', 'yes']:
                print("\n🔐 Email Authentication Required")
                print("Enter your email password for roy@agtpartners.com.sg")
                print("Note: Your password is only used locally and not stored anywhere.")
                
                try:
                    password = getpass.getpass("Password: ")
                    
                    if not password.strip():
                        print("❌ Password cannot be empty")
                    else:
                        # First test the connection
                        print("\n📋 Step 1: Testing email connection...")
                        connection_ok = test_email_connection(password)
                        
                        if connection_ok:
                            print("\n✅ Email connection successful!")
                            
                            # Ask for confirmation before full pipeline
                            confirm = input("\nProceed with full pipeline processing? (y/n): ").lower().strip()
                            
                            if confirm in ['y', 'yes']:
                                print("\n📋 Step 2: Running full email processing pipeline...")
                                success, output = run_pipeline_with_monitoring(password)
                                
                                if success:
                                    print("\n🔄 Step 3: Reloading dashboard data...")
                                    # Reinitialize dashboard to load new data
                                    dashboard = EmailDashboard()
                                    has_data = dashboard.load_all_data()
                                    
                                    if has_data:
                                        print("✅ New email data loaded successfully!")
                                    else:
                                        print("⚠️ Pipeline completed but no data was loaded. Check the pipeline logs above.")
                                else:
                                    print("❌ Pipeline execution failed. Please check the error messages above.")
                            else:
                                print("⏹️ Pipeline execution cancelled by user.")
                        else:
                            print("❌ Email connection failed. Please check your password and network connection.")
                            print("💡 You can also run the pipeline manually with:")
                            print("   python process_emails.py YOUR_PASSWORD")
                            
                except KeyboardInterrupt:
                    print("\n⏹️ Pipeline setup cancelled by user.")
                except Exception as e:
                    print(f"❌ Error during pipeline setup: {e}")
            else:
                print("💡 To run the pipeline manually:")
                print("   python process_emails.py YOUR_PASSWORD")
                
        except Exception as e:
            print(f"❌ Interactive input not available: {e}")
            is_interactive = False
    
    if not is_interactive:
        # Non-interactive mode - check for environment variable
        print("📋 Running in non-interactive mode")
        
        env_password = get_password_from_environment()
        if env_password:
            print("🔐 Found password in environment variable")
            print("🔄 Running pipeline automatically...")
            
            # Test connection first
            print("\n📋 Step 1: Testing email connection...")
            connection_ok = test_email_connection(env_password)
            
            if connection_ok:
                print("\n✅ Email connection successful!")
                print("\n📋 Step 2: Running full email processing pipeline...")
                success, output = run_pipeline_with_monitoring(env_password)
                
                if success:
                    print("\n🔄 Step 3: Reloading dashboard data...")
                    # Reinitialize dashboard to load new data
                    dashboard = EmailDashboard()
                    has_data = dashboard.load_all_data()
                    
                    if has_data:
                        print("✅ New email data loaded successfully!")
                    else:
                        print("⚠️ Pipeline completed but no data was loaded. Check the pipeline logs above.")
                else:
                    print("❌ Pipeline execution failed. Please check the error messages above.")
            else:
                print("❌ Email connection failed. Please check your password.")
        else:
            print("💡 To run the pipeline in non-interactive mode:")
            print("   1. Set environment variable: export EMAIL_PASSWORD='your_password'")
            print("   2. Or run manually: python process_emails.py YOUR_PASSWORD")
            print("   3. Then re-run this notebook to see your results")
            
            print("\n💡 To run the pipeline interactively:")
            print("   1. Open this notebook in Jupyter Lab/Notebook")
            print("   2. Run the cells to get interactive prompts")

else:
    print("\n🎉 Email data loaded successfully!")

🔍 Searching for processed email data...
📭 No pipeline databases found

⚠️ No processed email data found.
📋 Running in non-interactive mode
💡 To run the pipeline in non-interactive mode:
   1. Set environment variable: export EMAIL_PASSWORD='your_password'
   2. Or run manually: python process_emails.py YOUR_PASSWORD
   3. Then re-run this notebook to see your results

💡 To run the pipeline interactively:
   1. Open this notebook in Jupyter Lab/Notebook
   2. Run the cells to get interactive prompts


## 📊 Step 2: Email Processing Overview

High-level statistics about processed emails.

In [3]:
if has_data:
    # Email processing statistics
    total_emails = len(dashboard.emails_df)
    successful_emails = len(dashboard.emails_df[dashboard.emails_df['status'] == 'completed'])
    failed_emails = len(dashboard.emails_df[dashboard.emails_df['status'] == 'failed'])
    pending_emails = len(dashboard.emails_df[dashboard.emails_df['status'] == 'pending'])
    
    # Create overview dashboard
    fig = make_subplots(
        rows=2, cols=3,
        subplot_titles=('Processing Status', 'Email Priority Distribution', 'Processing Time',
                       'Daily Email Volume', 'Top Senders', 'Success Rate Trend'),
        specs=[[{'type': 'domain'}, {'type': 'bar'}, {'type': 'histogram'}],
               [{'type': 'bar'}, {'type': 'bar'}, {'type': 'scatter'}]]
    )
    
    # 1. Processing Status Pie Chart
    status_counts = dashboard.emails_df['status'].value_counts()
    fig.add_trace(
        go.Pie(
            labels=status_counts.index,
            values=status_counts.values,
            name="Status",
            marker_colors=['#2ecc71', '#e74c3c', '#f39c12']
        ),
        row=1, col=1
    )
    
    # 2. Priority Distribution
    priority_hist = dashboard.emails_df['priority'].value_counts().sort_index()
    fig.add_trace(
        go.Bar(
            x=priority_hist.index,
            y=priority_hist.values,
            name="Priority",
            marker_color='#3498db'
        ),
        row=1, col=2
    )
    
    # 3. Processing Time Distribution
    if 'processing_time_ms' in dashboard.emails_df.columns:
        processing_times = dashboard.emails_df['processing_time_ms'].dropna()
        if not processing_times.empty:
            fig.add_trace(
                go.Histogram(
                    x=processing_times / 1000,  # Convert to seconds
                    name="Processing Time (s)",
                    marker_color='#9b59b6'
                ),
                row=1, col=3
            )
    
    # 4. Daily Email Volume
    if 'processed_date' in dashboard.emails_df.columns:
        daily_counts = pd.to_datetime(dashboard.emails_df['processed_date']).dt.date.value_counts().sort_index()
        fig.add_trace(
            go.Bar(
                x=daily_counts.index,
                y=daily_counts.values,
                name="Daily Volume",
                marker_color='#1abc9c'
            ),
            row=2, col=1
        )
    
    # 5. Top Senders
    top_senders = dashboard.emails_df['sender'].value_counts().head(10)
    fig.add_trace(
        go.Bar(
            y=[sender[:30] + '...' if len(sender) > 30 else sender for sender in top_senders.index],
            x=top_senders.values,
            orientation='h',
            name="Sender Volume",
            marker_color='#e67e22'
        ),
        row=2, col=2
    )
    
    # 6. Success Rate Trend (if we have date data)
    if 'processed_date' in dashboard.emails_df.columns:
        dashboard.emails_df['date'] = pd.to_datetime(dashboard.emails_df['processed_date'])
        daily_success = dashboard.emails_df.groupby(dashboard.emails_df['date'].dt.date)['status'].apply(
            lambda x: (x == 'completed').sum() / len(x) * 100
        )
        
        fig.add_trace(
            go.Scatter(
                x=daily_success.index,
                y=daily_success.values,
                mode='lines+markers',
                name="Success Rate %",
                line_color='#27ae60'
            ),
            row=2, col=3
        )
    
    # Update layout
    fig.update_layout(
        height=800,
        showlegend=False,
        title_text="📧 Email Processing Dashboard Overview",
        title_x=0.5
    )
    
    fig.show()
    
    # Print summary statistics
    print("\n📊 PROCESSING SUMMARY")
    print("=" * 50)
    print(f"📧 Total emails processed: {total_emails:,}")
    print(f"✅ Successful: {successful_emails:,} ({successful_emails/total_emails*100:.1f}%)")
    print(f"❌ Failed: {failed_emails:,} ({failed_emails/total_emails*100:.1f}%)")
    print(f"⏳ Pending: {pending_emails:,} ({pending_emails/total_emails*100:.1f}%)")
    
    if 'processing_time_ms' in dashboard.emails_df.columns:
        avg_time = dashboard.emails_df['processing_time_ms'].mean() / 1000
        print(f"⏱️ Average processing time: {avg_time:.2f}s")
    
    print(f"📎 Total attachments: {len(dashboard.attachments_df):,}")
    print(f"📊 ICE integrations: {len(dashboard.ice_data):,}")

else:
    print("⚠️ No data to display. Please run the pipeline first.")

⚠️ No data to display. Please run the pipeline first.


## 📋 Step 3: Detailed Email List

Interactive table showing all processed emails with search and filtering.

In [4]:
if has_data:
    # Create a detailed email display
    display_df = dashboard.emails_df.copy()
    
    # Clean up the display
    if 'subject' in display_df.columns:
        display_df['subject'] = display_df['subject'].apply(
            lambda x: (x[:60] + '...') if pd.notna(x) and len(str(x)) > 60 else x
        )
    
    if 'sender' in display_df.columns:
        display_df['sender'] = display_df['sender'].apply(
            lambda x: (x[:40] + '...') if pd.notna(x) and len(str(x)) > 40 else x
        )
    
    # Select columns for display
    display_columns = ['email_uid', 'subject', 'sender', 'processed_date', 'status', 'priority']
    available_columns = [col for col in display_columns if col in display_df.columns]
    
    print("\n📧 PROCESSED EMAILS DETAILS")
    print("=" * 80)
    
    if available_columns:
        # Sort by processed date (most recent first)
        if 'processed_date' in display_df.columns:
            display_df = display_df.sort_values('processed_date', ascending=False)
        
        # Show top 20 emails
        print(display_df[available_columns].head(20).to_string(index=False, max_colwidth=50))
        
        if len(display_df) > 20:
            print(f"\n... and {len(display_df) - 20} more emails")
    
    # Show high priority emails
    high_priority = dashboard.emails_df[dashboard.emails_df['priority'] > 50]
    if not high_priority.empty:
        print("\n\n🔥 HIGH PRIORITY EMAILS")
        print("=" * 40)
        for _, email in high_priority.head(10).iterrows():
            subject = email.get('subject', 'No Subject')[:60]
            sender = email.get('sender', 'Unknown')[:30]
            priority = email.get('priority', 0)
            status = email.get('status', 'unknown')
            
            status_emoji = {'completed': '✅', 'failed': '❌', 'pending': '⏳'}.get(status, '❓')
            
            print(f"{status_emoji} Priority {priority}: {subject}")
            print(f"   📧 From: {sender}")
            print()
    
    # Show failed emails for troubleshooting
    failed_emails = dashboard.emails_df[dashboard.emails_df['status'] == 'failed']
    if not failed_emails.empty:
        print("\n\n❌ FAILED EMAILS (for troubleshooting)")
        print("=" * 50)
        for _, email in failed_emails.head(5).iterrows():
            subject = email.get('subject', 'No Subject')[:50]
            error = email.get('error_message', 'Unknown error')[:80]
            
            print(f"📧 {subject}")
            print(f"   ❌ Error: {error}")
            print()

else:
    print("⚠️ No email data available.")

⚠️ No email data available.


## 🎯 Step 4: Investment Entity Analysis

Analysis of extracted investment entities from processed emails.

In [5]:
if has_data and dashboard.ice_data:
    print("🎯 INVESTMENT ENTITY ANALYSIS")
    print("=" * 50)
    
    # Aggregate data from ICE analysis results
    all_tickers = []
    all_companies = []
    all_people = []
    
    for ice_result in dashboard.ice_data:
        if 'entities' in ice_result:
            entities = ice_result['entities']
            all_tickers.extend(entities.get('tickers', []))
            all_companies.extend(entities.get('companies', []))
            all_people.extend(entities.get('people', []))
    
    if all_tickers or all_companies:
        # Create entity visualization
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=('Top Mentioned Tickers', 'Top Mentioned Companies',
                           'Entity Type Distribution', 'Storage Statistics'),
            specs=[[{'type': 'bar'}, {'type': 'bar'}],
                   [{'type': 'domain'}, {'type': 'bar'}]]
        )
        
        # 1. Top Tickers
        if all_tickers:
            ticker_counts = pd.Series(all_tickers).value_counts().head(15)
            fig.add_trace(
                go.Bar(
                    x=ticker_counts.index,
                    y=ticker_counts.values,
                    name="Tickers",
                    marker_color='#3498db'
                ),
                row=1, col=1
            )
        
        # 2. Top Companies
        if all_companies:
            company_counts = pd.Series(all_companies).value_counts().head(10)
            fig.add_trace(
                go.Bar(
                    y=[comp[:25] + '...' if len(comp) > 25 else comp for comp in company_counts.index],
                    x=company_counts.values,
                    orientation='h',
                    name="Companies",
                    marker_color='#2ecc71'
                ),
                row=1, col=2
            )
        
        # 3. Entity Type Distribution
        entity_types = ['Tickers', 'Companies', 'People']
        entity_counts = [len(set(all_tickers)), len(set(all_companies)), len(set(all_people))]
        
        fig.add_trace(
            go.Pie(
                labels=entity_types,
                values=entity_counts,
                name="Entity Types"
            ),
            row=2, col=1
        )
        
        # 4. ICE Storage Statistics
        storage_stats = {}
        for ice_result in dashboard.ice_data:
            if 'storage_stats' in ice_result:
                for key, value in ice_result['storage_stats'].items():
                    storage_stats[key] = storage_stats.get(key, 0) + value
        
        if storage_stats:
            fig.add_trace(
                go.Bar(
                    x=list(storage_stats.keys()),
                    y=list(storage_stats.values()),
                    name="Storage",
                    marker_color='#e74c3c'
                ),
                row=2, col=2
            )
        
        fig.update_layout(
            height=800,
            showlegend=False,
            title_text="🎯 Investment Entity Analysis",
            title_x=0.5
        )
        
        fig.show()
        
        # Print detailed statistics
        print(f"\n📈 TICKER ANALYSIS:")
        print(f"   Total ticker mentions: {len(all_tickers)}")
        print(f"   Unique tickers: {len(set(all_tickers))}")
        
        if all_tickers:
            top_tickers = pd.Series(all_tickers).value_counts().head(10)
            print(f"   Top tickers: {', '.join(top_tickers.index.tolist())}")
        
        print(f"\n🏢 COMPANY ANALYSIS:")
        print(f"   Total company mentions: {len(all_companies)}")
        print(f"   Unique companies: {len(set(all_companies))}")
        
        print(f"\n👤 PEOPLE ANALYSIS:")
        print(f"   Total people mentions: {len(all_people)}")
        print(f"   Unique people: {len(set(all_people))}")
    
    else:
        print("📊 Entity extraction results not found in ICE data.")
        print("💡 This might indicate the emails were processed with a different version.")

else:
    print("⚠️ No ICE entity data available.")
    print("💡 Ensure the pipeline completed successfully and ICE integration worked.")

⚠️ No ICE entity data available.
💡 Ensure the pipeline completed successfully and ICE integration worked.


## 📎 Step 5: Attachment Analysis

Analysis of processed email attachments and document extraction.

In [6]:
if has_data and not dashboard.attachments_df.empty:
    print("📎 ATTACHMENT ANALYSIS")
    print("=" * 40)
    
    # Attachment statistics
    total_attachments = len(dashboard.attachments_df)
    processed_attachments = len(dashboard.attachments_df[dashboard.attachments_df['processing_status'] == 'completed'])
    
    print(f"📎 Total attachments: {total_attachments:,}")
    print(f"✅ Successfully processed: {processed_attachments:,} ({processed_attachments/total_attachments*100:.1f}%)")
    
    # File type analysis
    if 'mime_type' in dashboard.attachments_df.columns:
        file_types = dashboard.attachments_df['mime_type'].value_counts()
        
        print(f"\n📊 File Types:")
        for mime_type, count in file_types.head(10).items():
            percentage = count / total_attachments * 100
            print(f"   {mime_type}: {count:,} ({percentage:.1f}%)")
    
    # Size analysis
    if 'file_size' in dashboard.attachments_df.columns:
        total_size_mb = dashboard.attachments_df['file_size'].sum() / (1024 * 1024)
        avg_size_kb = dashboard.attachments_df['file_size'].mean() / 1024
        
        print(f"\n💾 Size Analysis:")
        print(f"   Total size: {total_size_mb:.1f} MB")
        print(f"   Average size: {avg_size_kb:.1f} KB")
    
    # OCR confidence analysis
    if 'ocr_confidence' in dashboard.attachments_df.columns:
        ocr_data = dashboard.attachments_df['ocr_confidence'].dropna()
        if not ocr_data.empty:
            avg_confidence = ocr_data.mean()
            print(f"\n🔍 OCR Analysis:")
            print(f"   Average confidence: {avg_confidence:.2f}")
            print(f"   High confidence (>0.8): {len(ocr_data[ocr_data > 0.8])} files")
    
    # Create attachment visualization
    fig = make_subplots(
        rows=1, cols=3,
        subplot_titles=('File Type Distribution', 'Processing Status', 'File Size Distribution'),
        specs=[[{'type': 'domain'}, {'type': 'domain'}, {'type': 'histogram'}]]
    )
    
    # File types pie chart
    if 'mime_type' in dashboard.attachments_df.columns:
        file_types = dashboard.attachments_df['mime_type'].value_counts().head(8)
        fig.add_trace(
            go.Pie(
                labels=[t.split('/')[-1] for t in file_types.index],  # Simplify mime types
                values=file_types.values,
                name="File Types"
            ),
            row=1, col=1
        )
    
    # Processing status pie chart
    status_counts = dashboard.attachments_df['processing_status'].value_counts()
    fig.add_trace(
        go.Pie(
            labels=status_counts.index,
            values=status_counts.values,
            name="Status"
        ),
        row=1, col=2
    )
    
    # File size histogram
    if 'file_size' in dashboard.attachments_df.columns:
        sizes_kb = dashboard.attachments_df['file_size'] / 1024
        fig.add_trace(
            go.Histogram(
                x=sizes_kb,
                name="Size (KB)",
                nbinsx=20
            ),
            row=1, col=3
        )
    
    fig.update_layout(
        height=400,
        showlegend=False,
        title_text="📎 Attachment Processing Analysis",
        title_x=0.5
    )
    
    fig.show()
    
    # Show sample attachments
    print("\n📋 SAMPLE ATTACHMENTS:")
    sample_attachments = dashboard.attachments_df.head(10)
    for _, att in sample_attachments.iterrows():
        filename = att.get('filename', 'Unknown')[:40]
        size_kb = att.get('file_size', 0) / 1024 if pd.notna(att.get('file_size')) else 0
        status = att.get('processing_status', 'unknown')
        method = att.get('extraction_method', 'unknown')
        
        status_emoji = {'completed': '✅', 'failed': '❌', 'pending': '⏳'}.get(status, '❓')
        
        print(f"{status_emoji} {filename} ({size_kb:.1f} KB) - {method}")

else:
    print("📎 No attachment data available.")
    print("💡 This is normal if processed emails didn't contain attachments.")

📎 No attachment data available.
💡 This is normal if processed emails didn't contain attachments.


## 🔍 Step 6: Interactive Email Search

Search through processed emails by various criteria.

In [7]:
if has_data:
    print("🔍 INTERACTIVE EMAIL SEARCH")
    print("=" * 40)
    
    def search_emails(query="", sender="", priority_min=0, status="all"):
        """Search emails with various filters"""
        df = dashboard.emails_df.copy()
        
        # Apply filters
        if query:
            mask = df['subject'].str.contains(query, case=False, na=False)
            df = df[mask]
        
        if sender:
            mask = df['sender'].str.contains(sender, case=False, na=False)
            df = df[mask]
        
        if priority_min > 0:
            df = df[df['priority'] >= priority_min]
        
        if status != "all":
            df = df[df['status'] == status]
        
        return df
    
    # Example searches
    print("📧 Example searches:")
    print()
    
    # 1. High priority emails
    high_priority = search_emails(priority_min=50)
    print(f"🔥 High priority emails (>50): {len(high_priority)}")
    if not high_priority.empty:
        for _, email in high_priority.head(3).iterrows():
            subject = email.get('subject', 'No Subject')[:50]
            priority = email.get('priority', 0)
            print(f"   • Priority {priority}: {subject}")
    print()
    
    # 2. Search for investment terms
    investment_terms = ['earnings', 'portfolio', 'analysis', 'rating', 'target']
    
    for term in investment_terms:
        results = search_emails(query=term)
        if not results.empty:
            print(f"📈 Emails mentioning '{term}': {len(results)}")
            
            # Show top result
            if len(results) > 0:
                top_result = results.iloc[0]
                subject = top_result.get('subject', 'No Subject')[:60]
                sender = top_result.get('sender', 'Unknown')[:30]
                print(f"   📧 {subject} (from {sender})")
            print()
    
    # 3. Search by sender domain
    sender_domains = ['research', 'analyst', 'investment', 'agtpartners']
    
    print("👥 Emails by sender type:")
    for domain in sender_domains:
        results = search_emails(sender=domain)
        if not results.empty:
            print(f"   {domain}: {len(results)} emails")
    print()
    
    # 4. Failed emails for troubleshooting
    failed = search_emails(status='failed')
    print(f"❌ Failed emails: {len(failed)}")
    if not failed.empty:
        print("   Common failure patterns:")
        if 'error_message' in failed.columns:
            error_counts = failed['error_message'].value_counts().head(3)
            for error, count in error_counts.items():
                if pd.notna(error):
                    error_short = error[:60] + '...' if len(str(error)) > 60 else error
                    print(f"     • {error_short}: {count} occurrences")
    
    print("\n💡 You can create custom searches using the search_emails() function:")
    print("   search_emails(query='NVIDIA', priority_min=30)")
    print("   search_emails(sender='research', status='completed')")

else:
    print("⚠️ No email data available for searching.")

⚠️ No email data available for searching.


## 💭 Step 7: Query Interface

Interface to query the processed email knowledge base.

In [8]:
if has_data:
    print("💭 ICE QUERY INTERFACE")
    print("=" * 40)
    
    # Try to load ICE integrator from the working directories
    ice_integrator = None
    
    for working_dir in dashboard.working_dirs:
        ice_storage_dir = os.path.join(working_dir, 'ice_storage')
        if os.path.exists(ice_storage_dir):
            try:
                # Import ICE integrator
                from ice_integrator import ICEEmailIntegrator
                ice_integrator = ICEEmailIntegrator(ice_storage_dir)
                print(f"✅ Connected to ICE knowledge base: {ice_storage_dir}")
                break
            except Exception as e:
                print(f"⚠️ Could not load ICE integrator: {e}")
                continue
    
    if ice_integrator:
        # Sample queries
        sample_queries = [
            "What stocks were mentioned in my recent emails?",
            "What's the sentiment around technology stocks?",
            "Who are the key analysts mentioned?",
            "What companies have earnings updates?",
            "What investment risks were highlighted?"
        ]
        
        print("\n📝 Sample queries you can ask:")
        for i, query in enumerate(sample_queries, 1):
            print(f"   {i}. {query}")
        
        print("\n🔍 Query Examples:")
        
        # Try a few sample queries
        test_queries = [
            "What tickers were mentioned?",
            "Show me investment recommendations"
        ]
        
        for query in test_queries:
            print(f"\n❓ Query: {query}")
            try:
                # Note: This might need async handling depending on implementation
                result = ice_integrator.query_email_content(query, mode="hybrid")
                
                if isinstance(result, dict) and result.get('success'):
                    response = result.get('response', 'No response generated')
                    print(f"✅ Response: {response[:200]}..." if len(response) > 200 else f"✅ Response: {response}")
                else:
                    print(f"❌ Query failed or returned no results")
                    
            except Exception as e:
                print(f"⚠️ Query error: {e}")
        
        print("\n💡 To run custom queries:")
        print("   result = ice_integrator.query_email_content('Your question here')")
        print("   print(result)")
        
    else:
        print("❌ ICE knowledge base not found")
        print("💡 Make sure you ran the full pipeline: python process_emails.py")
        
        # Show available data summary instead
        if dashboard.ice_data:
            print("\n📊 Available ICE data summary:")
            for i, ice_result in enumerate(dashboard.ice_data[:3], 1):
                print(f"   Source {i}:")
                if 'entities' in ice_result:
                    entities = ice_result['entities']
                    print(f"     Tickers: {len(entities.get('tickers', []))}")
                    print(f"     Companies: {len(entities.get('companies', []))}")
                if 'summary' in ice_result:
                    summary = ice_result['summary']
                    print(f"     Processed: {summary.get('processed_count', 0)} emails")

else:
    print("⚠️ No processed email data available for querying.")

⚠️ No processed email data available for querying.


## 📊 Step 8: Export and Summary

Export processed data and generate final summary report.

In [9]:
if has_data:
    print("📊 EXPORT AND SUMMARY")
    print("=" * 40)
    
    # Generate export files
    export_dir = "./email_analysis_exports"
    os.makedirs(export_dir, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Export emails to CSV
    emails_export = os.path.join(export_dir, f"processed_emails_{timestamp}.csv")
    dashboard.emails_df.to_csv(emails_export, index=False)
    print(f"✅ Emails exported to: {emails_export}")
    
    # Export attachments to CSV
    if not dashboard.attachments_df.empty:
        attachments_export = os.path.join(export_dir, f"processed_attachments_{timestamp}.csv")
        dashboard.attachments_df.to_csv(attachments_export, index=False)
        print(f"✅ Attachments exported to: {attachments_export}")
    
    # Export ICE data summary
    if dashboard.ice_data:
        ice_summary_export = os.path.join(export_dir, f"ice_data_summary_{timestamp}.json")
        with open(ice_summary_export, 'w') as f:
            json.dump(dashboard.ice_data, f, indent=2, default=str)
        print(f"✅ ICE data exported to: {ice_summary_export}")
    
    # Generate final summary report
    summary_report = {
        "generated_at": datetime.now().isoformat(),
        "pipeline_statistics": {
            "total_emails": len(dashboard.emails_df),
            "successful_emails": len(dashboard.emails_df[dashboard.emails_df['status'] == 'completed']) if 'status' in dashboard.emails_df.columns else 0,
            "failed_emails": len(dashboard.emails_df[dashboard.emails_df['status'] == 'failed']) if 'status' in dashboard.emails_df.columns else 0,
            "total_attachments": len(dashboard.attachments_df),
            "ice_integrations": len(dashboard.ice_data)
        },
        "top_senders": dashboard.emails_df['sender'].value_counts().head(10).to_dict() if 'sender' in dashboard.emails_df.columns else {},
        "priority_distribution": dashboard.emails_df['priority'].describe().to_dict() if 'priority' in dashboard.emails_df.columns else {},
        "working_directories": dashboard.working_dirs
    }
    
    summary_export = os.path.join(export_dir, f"pipeline_summary_{timestamp}.json")
    with open(summary_export, 'w') as f:
        json.dump(summary_report, f, indent=2, default=str)
    print(f"✅ Summary report exported to: {summary_export}")
    
    print("\n🎉 EMAIL ANALYSIS DASHBOARD COMPLETE!")
    print("=" * 50)
    print(f"📧 Total emails analyzed: {len(dashboard.emails_df):,}")
    
    if 'status' in dashboard.emails_df.columns:
        success_count = len(dashboard.emails_df[dashboard.emails_df['status'] == 'completed'])
        print(f"📊 Success rate: {success_count / len(dashboard.emails_df) * 100:.1f}%")
    
    print(f"📎 Attachments processed: {len(dashboard.attachments_df):,}")
    print(f"🧠 ICE integrations: {len(dashboard.ice_data):,}")
    print(f"💾 Export directory: {export_dir}")
    
    print("\n🔍 Next steps:")
    print("   • Use the query interface to ask questions about your emails")
    print("   • Review failed emails for troubleshooting")
    print("   • Run additional pipeline cycles to process more emails")
    print("   • Integrate with the main ICE system for advanced analysis")

else:
    print("⚠️ No data available to export.")
    print("\n🚀 To get started:")
    print("   1. Re-run this notebook - it will automatically prompt you to run the pipeline")
    print("   2. Or manually run: python process_emails.py YOUR_PASSWORD")
    print("   3. Then re-run this notebook to see your results")
    print("\n💡 This dashboard will automatically:")
    print("   • Test your email connection first") 
    print("   • Show real-time pipeline progress")
    print("   • Automatically refresh data after completion")
    print("   • Handle errors gracefully with clear instructions")

⚠️ No data available to export.

🚀 To get started:
   1. Re-run this notebook - it will automatically prompt you to run the pipeline
   2. Or manually run: python process_emails.py YOUR_PASSWORD
   3. Then re-run this notebook to see your results

💡 This dashboard will automatically:
   • Test your email connection first
   • Show real-time pipeline progress
   • Automatically refresh data after completion
   • Handle errors gracefully with clear instructions
